<a href="https://colab.research.google.com/github/shmanth/Ad_Placement_Optimization_Using_Reinforcement_learning/blob/master/Ad_Placement_Optimization_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predicting the ad positioning using Q-Learning





Placement of ads on website is the primary problem for companies that operate on ad revenue. The position where the ad is placed plays pivotal role on whether or not the ad will be clicked. Here we have the following choices:

Place them randomly, or
Place the ad on the same position
The problem with placing the ad on the same position is the user, after a certain time, will start ignoring the space since he's used to seeing ad at the place, he will end up ignoring that particular position hereafter. Hence, this will reduce the number of clicks on ads. The problem with the former option, placing them randomly, is it wouldn't take optimal positions into consideration. For instance, text beside images are viewed higher number of times than those text which are placed at a distance. It is infeasible to go through every website and repeat the procedure.

Solution: Reinforcement Learning

Using Reinforcement Learning we can approximate the human behavior.

Why Reinforcement Learning?
We cannot use traditional Machine Learning here, since it requires:

Huge data
Features
Tuning of many hyperparameters
And we neither have huge data, nor features. The only data we have is the position of the banner/ad and whether or not it was clicked. We will use this dataset from Kaggle: https://www.kaggle.com/akram24/ads-ctr-optimisation. We will solve this problem using both model-based (Policy iteration) and model-free methods(Q-Learning & Monte-Carlo). We'll simplify some assumptions for model-based technique.

Notebook Layout
MDP environment - Understanding the dataset
Random Policy - placing the ads randomly on different webpages
Max policy - placing the ad where it is clicked maximum number of times
Model-based Method - Policy Iteration
Model-free Method - Q-learning & Monte Carlo

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time

Dataset
Our environment will be the dataset. It contains 10 ads position per row having values either 1, when the ad is clicked, or 0 when it is not. Every row can be considered as a state in the space, considering it kind of a navigation across multiple pages (on website, for instance) Lets load the dataset and visualize the first few rows.

state = webpage
action = placing the ad at any of the 10 positions on a webpage
reward = +1 if the ad was clicked at the position; else 0
Transition Probability: next webpage that user will end up in is random; therefore, it is 1/(total_webpages -1)

In [2]:
data= pd.read_csv("/content/drive/MyDrive/Data_colab_practice_august_2023/Ads_CTR_Optimisation.csv")
data.head()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0


### Random policy -
If we were to not have Reinforcement Learning, we would place the ads randomly at given positions. We will now simulate the same

